In [1]:
import dotenv, os
dotenv.load_dotenv("../.env")
import google.generativeai as genai
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

/media/gamedisk/BaiTap/Code/Nam3/Ki1/AI/re-view/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pydantic import BaseModel, Field
from enum import Enum

In [3]:
SYSTEM_PROMPT_BASE = """Your name is Hải, your gender is male. You are a senior developer at {company_name}, with over 10 years of experience in software development. You are known for your expertise in {technology_and_domain} and your ability to lead projects successfully. Today, you are conducting an interview for the position of {job_title} as described in the following job description: {job_description}. Your goal is to assess the candidate's technical skills, experience, problem-solving abilities, and cultural fit within the team.

For this interview, please cover the following areas:
1. Technical Skills: Assess the candidate's proficiency in the required technologies and tools mentioned in the job description. Ask specific questions related to their experience with these technologies.
2. Problem-Solving: Present the candidate with a technical problem or scenario relevant to the job and evaluate their approach to solving it.
3. Experience: Inquire about the candidate's previous projects and roles, focusing on responsibilities and achievements that align with the job requirements.
4. Teamwork and Collaboration: Since our team values collaboration, ask questions that gauge the candidate's ability to work effectively in a team environment.
5. Adaptability: Given the ever-evolving tech landscape, assess how the candidate handles new technologies and changes in project requirements.

Approach the interview with empathy and professionalism. Remember that the candidate may be nervous, so create a welcoming and comfortable environment. Listen carefully to their responses and ask clarifying questions when needed.

Ensure that all questions are relevant to the job description and are asked in a neutral manner. Avoid any questions that could be considered discriminatory or invasive of the candidate's privacy. Focus solely on the candidate's qualifications and fit for the position.

At the end of the interview, summarize the main points discussed. Thank the candidate for their time and interest in the position."""

In [4]:
EXTRACT_JD_SYSTEM_PROMPT = """You are a senior developer with extensive experience in the IT industry. Your task is to analyze given IT job descriptions and extract key information to help job seekers and recruiters make informed decisions. Specifically, you need to identify the company name, job title, and the technologies and domains mentioned in the job description.

Your output should be in JSON format, containing the following fields:
- `companyName`: The name of the company as stated in the job description. This should be a string, or null if the company name is not provided.
- `jobTitle`: The title of the job being described. This should be a string, or null if not specified.
- `domain`: An array of strings, each representing a domain area mentioned in the job description.

As a senior developer, you should be adept at interpreting job descriptions, understanding technical requirements, and identifying relevant"""


class JD_Extracted(BaseModel):
    companyName: str | None
    jobTitle: str | None
    domain: list[str]


def extractJD(jd: str) -> JD_Extracted:
    if not hasattr(extractJD, "model"):
        extractJD.model = genai.GenerativeModel(system_instruction=EXTRACT_JD_SYSTEM_PROMPT)

    return JD_Extracted.model_validate_json(
        extractJD.model.generate_content(
            jd,
            generation_config={
                "response_mime_type": "application/json",
                "temperature": 0.5
            },
        ).text
    )



In [5]:
JOB_DESCRIPTION = """Mô tả công việc
LG CNS is looking for full stack developers for cloud domain project

Develop software/web applicant
Business Analyst
Cooperate between HQ and VNB

Yêu cầu ứng viên
[Required]

Bachelor's degree of Information Technology or higher
Have working experiment and excellent knowledge at software developing using Java, Spring boot
Have working experiment and excellent knowledge at software developing using React, HTML, JavaScript
Good knowledge about AI (Azure OpenAI and GenAI)
Good knowledge about public cloud (Azure, AWS,)
Database: MariaDB (or MySQL).
[Preferred]

Having experiment with Python and Google cloud"""

# extractJD(JOB_DESCRIPTION)
JOB_DESCRIPTION

"Mô tả công việc\nLG CNS is looking for full stack developers for cloud domain project\n\nDevelop software/web applicant\nBusiness Analyst\nCooperate between HQ and VNB\n\nYêu cầu ứng viên\n[Required]\n\nBachelor's degree of Information Technology or higher\nHave working experiment and excellent knowledge at software developing using Java, Spring boot\nHave working experiment and excellent knowledge at software developing using React, HTML, JavaScript\nGood knowledge about AI (Azure OpenAI and GenAI)\nGood knowledge about public cloud (Azure, AWS,)\nDatabase: MariaDB (or MySQL).\n[Preferred]\n\nHaving experiment with Python and Google cloud"

In [6]:
class NextAction(str, Enum):
    continueOnSameTopic = "Continue on the same topic"
    changeTopic = "Switch to a new topic"
    startCodingInterview = "Switch to coding interview"


class AnalysisAndNextActionOutput(BaseModel):
    analysis: str
    score: float = Field(ge=0, le=10)
    nextAction: NextAction
    nextQuestion: str


class AnalysisAndNextActionInput(BaseModel):
    job_description: str
    conversation: list[dict[str, str]]


ANALYSIS_AND_NEXT_ACTION_SYSTEM_PROMPT = """You are a highly experienced senior developer participating in technical interviews. Your goal is to assist the human interviewer in evaluating candidates. You will be provided with the job description for the role and the ongoing conversation between the interviewer and the candidate. After each candidate response, you need to analyze the answer, provide a score reflecting your assessment, and suggest the next action for the human interviewer. Your analysis should consider the key areas of the interview: Technical Skills, Problem-Solving, Experience, Teamwork and Collaboration, and Adaptability, as defined in the job description. Be objective and justify your score and next action recommendation based on the provided information. Focus on the relevance of the candidate's answer to the job requirements and the overall quality of their response. If the candidate is asked a direct technical question, assess their technical accuracy and depth of understanding. For problem-solving scenarios, analyze their approach, clarity of thought, and consideration of edge cases. When discussing experience, evaluate the alignment of their past roles and responsibilities with the job requirements and their ability to articulate their contributions and learnings. For teamwork questions, assess their understanding of collaborative principles and their ability to work effectively in a team. For adaptability, consider their openness to learning new technologies and their experience with handling change. Remember that your primary responsibility is to provide insightful and actionable feedback to aid the interview process. Do not engage in conversation with the candidate; your output is solely for the human interviewer's benefit.

Input format: 
{
    "job_description": "{{job_description_text}}",
    "conversation": [
        {"interviewer": "{{interviewer_question}}"},
        {"candidate": "{{candidate_answer}}"}
        // ... more turns of conversation
    ]
}
Output format: 
{
    "analysis": "Detailed analysis of the candidate's answer, referencing the evaluation areas.",
    "score": "A float between 0.0 and 10.0, where 0.0 is completely inadequate and 10.0 is exceptional.",
    "nextAction": "One of: 'Continue on the same topic', 'Switch to a new topic', 'Switch to coding interview'",
    "nextQuestion": "A recommended question for the interviewer"
}
Analysis Guidelines:
1. Analyze the candidate's response in the context of the job description.
2. Justify your score by highlighting specific strengths and weaknesses in the candidate's answer.
3. Recommend the next action based on the information gained from the candidate's response and the overall interview plan.
4. If the candidate's answer is incomplete or unclear, suggest continuing on the same topic to probe further.
5. If the candidate has adequately addressed the current topic, suggest switching to a new topic to cover other evaluation areas.
6. If the candidate has demonstrated sufficient technical aptitude and problem-solving skills, suggest switching to a coding interview to assess their practical coding abilities.

Additional Note:
1. Consider the flow of the conversation and the conversation length when suggesting the next action.
2. Be mindful of the seniority level of the role when evaluating the candidate's responses.
3. Ensure your analysis and recommendations are objective and free from personal biases.
4. Coding interview should be the last thing in the interview process.
5. Your analysis and next question must be in the same language as the conversation"""


def getAnalysisAndNextAction(jd: str, history: list[tuple[str, str]]):
    if not hasattr(getAnalysisAndNextAction, "model"):
        getAnalysisAndNextAction.model = genai.GenerativeModel(
            system_instruction=ANALYSIS_AND_NEXT_ACTION_SYSTEM_PROMPT
        )
    conv = []
    for question, answer in history:
        conv.append({"interviewer": question})
        conv.append({"candidate": answer})
    return AnalysisAndNextActionOutput.model_validate_json(
        getAnalysisAndNextAction.model.generate_content(
            AnalysisAndNextActionInput(
                job_description=jd, conversation=conv
            ).model_dump_json(),
            generation_config={"response_mime_type": "application/json", "temperature": 0.5},
        ).text
    )

In [7]:
AnalysisAndNextActionOutput.model_validate_json('{"analysis": "slfkdjsaf", "score": 9.5, "nextAction": "Continue on the same topic", "nextQuestion": "sldfkjsaldf"}')

AnalysisAndNextActionOutput(analysis='slfkdjsaf', score=9.5, nextAction=<NextAction.continueOnSameTopic: 'Continue on the same topic'>, nextQuestion='sldfkjsaldf')

In [8]:
getAnalysisAndNextAction(
    jd=JOB_DESCRIPTION,
    history=[
        ("Trong mô tả công việc, chúng tôi có đề cập đến việc sử dụng Java, Spring Boot, React, HTML, và JavaScript. Bạn có thể chia sẻ kinh nghiệm của mình với các công nghệ này được không? Cụ thể, bạn đã sử dụng chúng trong những dự án nào và vai trò của bạn trong các dự án đó là gì?",
        "Tôi đã có nhiều kinh nghiệm với Java và Spring Boot trong các dự án backend. Với Java, tôi đã phát triển các ứng dụng doanh nghiệp, đặc biệt là các hệ thống quản lý tài chính và thương mại điện tử. Spring Boot là công cụ chính tôi sử dụng để tạo các ứng dụng web dựa trên kiến trúc Microservices. Trong các dự án này, tôi đã thiết kế và triển khai các API RESTful, sử dụng Spring Security để bảo mật, Spring Data JPA để tương tác với cơ sở dữ liệu, và Spring Cloud để triển khai các dịch vụ phân tán. Một trong những dự án đáng chú ý là phát triển một hệ thống quản lý bán hàng với các tính năng như quản lý sản phẩm, đơn hàng và thanh toán trực tuyến. Trong các dự án sử dụng Spring Boot, tôi đảm nhận vai trò Backend Developer, nơi tôi thiết kế các API, xử lý logic nghiệp vụ và tối ưu hiệu suất của hệ thống."),
    ],
).model_dump(mode='python')

{'analysis': "The candidate demonstrates good experience with Java and Spring Boot, aligning well with the job requirements.  They provide specific examples of projects and their roles, showcasing their understanding of backend development concepts like RESTful APIs, Spring Security, Spring Data JPA, and Spring Cloud.  The description of the sales management system project is relevant and provides a concrete example of their skills. However, the candidate hasn't mentioned their experience with React, HTML, and JavaScript, which are also crucial for the full-stack role.  The answer lacks details on the scale and complexity of the projects, and the impact of their contributions. The response is strong in terms of Technical Skills and Experience but lacks information on other key areas like Teamwork and Collaboration, Adaptability, and Problem-Solving.",
 'score': 7.5,
 'nextAction': <NextAction.continueOnSameTopic: 'Continue on the same topic'>,
 'nextQuestion': 'Bạn có thể chia sẻ thêm 

In [9]:
getAnalysisAndNextAction(
    jd=JOB_DESCRIPTION,
    history=[
        ("Trong mô tả công việc, chúng tôi có đề cập đến việc sử dụng Java, Spring Boot, React, HTML, và JavaScript. Bạn có thể chia sẻ kinh nghiệm của mình với các công nghệ này được không? Cụ thể, bạn đã sử dụng chúng trong những dự án nào và vai trò của bạn trong các dự án đó là gì?",
        "Tôi đã có nhiều kinh nghiệm với Java và Spring Boot trong các dự án backend. Với Java, tôi đã phát triển các ứng dụng doanh nghiệp, đặc biệt là các hệ thống quản lý tài chính và thương mại điện tử. Spring Boot là công cụ chính tôi sử dụng để tạo các ứng dụng web dựa trên kiến trúc Microservices. Trong các dự án này, tôi đã thiết kế và triển khai các API RESTful, sử dụng Spring Security để bảo mật, Spring Data JPA để tương tác với cơ sở dữ liệu, và Spring Cloud để triển khai các dịch vụ phân tán. Một trong những dự án đáng chú ý là phát triển một hệ thống quản lý bán hàng với các tính năng như quản lý sản phẩm, đơn hàng và thanh toán trực tuyến. Trong các dự án sử dụng Spring Boot, tôi đảm nhận vai trò Backend Developer, nơi tôi thiết kế các API, xử lý logic nghiệp vụ và tối ưu hiệu suất của hệ thống."),
        ("Tiếp theo, về phía frontend, bạn có thể chia sẻ thêm về kinh nghiệm của mình với React, HTML và JavaScript được không? Bạn đã sử dụng chúng trong những dự án nào và bạn có thể mô tả cụ thể hơn về vai trò của mình trong các dự án đó?", "Tôi có kinh nghiệm mạnh mẽ trong việc phát triển frontend sử dụng React và JavaScript. Tôi đã tham gia vào việc xây dựng giao diện người dùng cho các ứng dụng web động với React, kết hợp với Redux để quản lý trạng thái toàn cục của ứng dụng. Tôi đã sử dụng HTML để tạo cấu trúc trang web và CSS/SCSS để định dạng giao diện. Bên cạnh đó, JavaScript được sử dụng để xử lý các sự kiện người dùng và tương tác với API từ backend, giúp tạo ra các trải nghiệm người dùng mượt mà. Một dự án tiêu biểu là phát triển một hệ thống quản lý khách hàng (CRM) với các tính năng như tìm kiếm nâng cao, biểu đồ thống kê và thông báo. Trong các dự án này, tôi đóng vai trò là Frontend Developer. Công việc của tôi bao gồm việc xây dựng các component React, xử lý state management, và đảm bảo các tính năng hoạt động trơn tru khi người dùng tương tác với giao diện.")
    ],
).model_dump(mode='python')

{'analysis': "The candidate demonstrates a good understanding and practical experience with Java, Spring Boot, React, HTML, and JavaScript.  Their descriptions of projects and roles are clear and specific, showcasing their abilities in backend and frontend development. They effectively highlight their use of key frameworks and technologies like Spring Security, Spring Data JPA, Spring Cloud, and Redux, indicating a solid grasp of architectural principles and best practices. The responses directly address the requirements outlined in the job description. While the candidate provides specific examples,  a deeper dive into the challenges faced and solutions implemented in those projects would strengthen their responses and showcase problem-solving abilities more effectively.  The candidate's answers are relevant and well-structured, though they could benefit from adding more quantitative details about project size, team size, and specific accomplishments.",
 'score': 8.0,
 'nextAction': <